In [ ]:
# 신발 검색
# 블로그가 뜸
# 블로그 정해진 개수만큼 가져와서 블로그별로 감정분석
# 블로그url을 인자로 감정분석 함수호출
# social_media_mentions = [model, positive, negative, neutral] 로 저장 
# 위에는 내가 밑에는 지피티가 함 

In [5]:
import time

In [ ]:
#!/usr/bin/env python3
#-*- coding: utf-8 -*-
import sys
import requests
import json
import nltk
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request

# 상위 디렉토리에서 config.py import
sys.path.append('..')
try:
    from config import (
        NAVER_SEARCH_CLIENT_ID, 
        NAVER_SEARCH_CLIENT_SECRET,
        NCP_CLIENT_ID,
        NCP_CLIENT_SECRET
    )
    # 검색용 api 인증키
    search_client_id = NAVER_SEARCH_CLIENT_ID
    search_client_secret = NAVER_SEARCH_CLIENT_SECRET
    # 감성분석용 api 인증키
    client_id = NCP_CLIENT_ID
    client_secret = NCP_CLIENT_SECRET
except ImportError:
    print("config.py 파일이 없습니다. config.example.py를 복사하여 config.py를 만들고 API 키를 입력하세요.")
    print("또는 아래 주석을 해제하여 직접 입력하세요.")
    # search_client_id = 'your_naver_search_client_id'
    # search_client_secret = 'your_naver_search_client_secret'
    # client_id = "your_ncp_client_id"
    # client_secret = "your_ncp_client_secret"
    raise

nltk.download('punkt', quiet=True)  # 이미 다운로드된 경우 조용히 넘어갑니다.
# api url 
analyze_url = "https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze" # 감정분석 api url
search_url = "https://openapi.naver.com/v1/search/blog?query=" # 신발이름으로 검색할거 url
start = 1
display = 20

headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/json"
}

# 네이버 블로그 본문 가져오기
def get_naver_blog_content(blog_url):
    try:
        response = requests.get(blog_url)
        if response.status_code != 200:
            print(f"Failed to fetch blog URL {blog_url} with status code {response.status_code}")
            return None
        
        soup = BeautifulSoup(response.text, "html.parser")
        # iframe URL 추출
        iframe = soup.find("iframe", {"id": "mainFrame"})
        if not iframe:
            print(f"No iframe found in blog URL {blog_url}")
            return None
        
        iframe_url = "https://blog.naver.com" + iframe["src"]
        response = requests.get(iframe_url)
        if response.status_code != 200:
            print(f"Failed to fetch iframe URL {iframe_url} with status code {response.status_code}")
            return None
        
        iframe_soup = BeautifulSoup(response.text, "html.parser")
        paragraphs = iframe_soup.find_all("p")

        return paragraphs

    except Exception as e:
        print(f"Exception occurred while fetching blog content: {e}")
        return None
        

# 블로그 글 감정 분석하기 
def analyze_emot(blog_url):
    paragraphs = get_naver_blog_content(blog_url)
    if not paragraphs:
        print(f"No paragraphs found in blog URL {blog_url}")
        return None

    # 블로그 글의 문장 리스트
    sentences = []
    for p in paragraphs:
        sentences.extend(nltk.sent_tokenize(p.get_text()))

    if not sentences:
        print(f"No sentences found in blog URL {blog_url}")
        return None

    # 블로그 글을 청크로 나누기 (문장 수 기준)
    chunk_size = 10
    chunks = [" ".join(sentences[i:i + chunk_size]) for i in range(0, len(sentences), chunk_size)]

    # 각 청크에 대해 감정 분석 수행
    sentiments = []
    for chunk in chunks:
        data = {
            "content": chunk
        }
        response = requests.post(analyze_url, data=json.dumps(data), headers=headers)
        if response.status_code == 200:
            result = json.loads(response.text)
            sentiment = result['document']['sentiment']
            sentiments.append(sentiment)
        else:
            print(f"Error in sentiment analysis API: {response.status_code} - {response.text}")
    
    # 전체 글의 감정 판단 (가장 많은 감정으로 글의 감정을 결정)
    positive_count = sentiments.count('positive')
    negative_count = sentiments.count('negative')
    neutral_count = sentiments.count('neutral')
    print(f"positive_count: {positive_count}\nnegative_count: {negative_count}\nneutral_count: {neutral_count}")

    if positive_count >= negative_count and positive_count >= neutral_count:
        return 'positive'
    elif negative_count > positive_count and negative_count >= neutral_count:
        return 'negative'
    else:
        return 'neutral'

# 신발별 감정분석 결과 
social_media_mentions = []  # model, positive, negative, neutral

# 신발가져오기
model_list = pd.read_csv('../dataset/data/RELEASE_PRICE.csv', encoding='utf-8')
model_list = model_list['model'][70:]

for model in model_list:
    print(f"Model: {model}")
    social_media_mention = [model, 0, 0, 0]
    
    encText = urllib.parse.quote(model)
    url = f"{search_url}{encText}&start={start}&display={display}"
    
    try:
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id", search_client_id)
        request.add_header("X-Naver-Client-Secret", search_client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read()
            response_json = json.loads(response_body)
        else:
            print(f"Error Code: {rescode}")
            continue
        
        items = response_json['items']
        for item in items:
            title = item['title']
            link = item['link']
            
            print(f"Blog Title: {title}")

            # 블로그 내용을 가져와서 감정 분석
            sentiment = analyze_emot(link)
            if sentiment == 'positive':
                social_media_mention[1] += 1
            elif sentiment == 'negative':
                social_media_mention[2] += 1
            elif sentiment == 'neutral':
                social_media_mention[3] += 1
            print(social_media_mention)
    except Exception as e:
        print(f"Exception occurred while fetching search results for model {model}: {e}")
    
    social_media_mentions.append(social_media_mention)

print(social_media_mentions)
'''
# 결과를 DataFrame으로 변환하여 CSV 파일로 저장
df_results = pd.DataFrame(social_media_mentions, columns=['Model', 'Positive', 'Negative', 'Neutral'])
df_results.to_csv('sentiment_analysis_results.csv', index=False, encoding='utf-8')
'''

Model: 푸마 스피드캣 OG 스파르코 블랙 화이트
Blog Title: <b>푸마 스피드캣 OG 스파르코 블랙 화이트</b> 리뷰 ft. 코디 사이즈 팁
positive_count: 4
negative_count: 1
neutral_count: 4
['푸마 스피드캣 OG 스파르코 블랙 화이트', 1, 0, 0]
Blog Title: 이게 돌아올줄이야 <b>푸마 스피드캣</b> LS / <b>OG 스파르코</b> 1월24일... 
positive_count: 4
negative_count: 2
neutral_count: 12
['푸마 스피드캣 OG 스파르코 블랙 화이트', 1, 0, 1]
Blog Title: 화려한 귀환 <b>푸마 스피드캣 OG 스파르코</b> &amp; 퓨마 <b>스피드</b>켓 LS... 
positive_count: 11
negative_count: 2
neutral_count: 7
['푸마 스피드캣 OG 스파르코 블랙 화이트', 2, 0, 1]
Blog Title: 2024트렌드 <b>푸마 스피드캣OG 스파르코 블랙</b> 사이즈... 
positive_count: 9
negative_count: 0
neutral_count: 5
['푸마 스피드캣 OG 스파르코 블랙 화이트', 3, 0, 1]
Blog Title: <b>푸마 스피드캣 OG</b> + <b>스파르코  블랙</b> 리뷰 및 코디 사이즈 팁... 


In [2]:

# 결과를 CSV 파일로 저장
result_df = pd.DataFrame(social_media_mentions, columns=['model', 'positive', 'negative', 'neutral'])
result_df.to_csv('./SOCIAL_MEDIA_MENTIONS.csv', mode='a', header=False, index=False, encoding='utf-8')

In [3]:
print(result_df)

                                                model  positive  negative  \
0                       나이키 P-6000 프리미엄 한글날 포톤 더스트 카키        21         0   
1                       살로몬 RX 슬라이드 3.0 아몬드 밀크 알로에 워시        23         0   
2                                    아디다스 슈퍼스타 트리플 블랙        49         3   
3                            나이키 에어포스 1 미드 '07 WB 플랙스        46         5   
4                      조던 1 x 트래비스 스캇 레트로 하이 OG SP 모카        43         3   
5                            조던 1 레트로 하이 스파이더맨 넥스트 챕터        41         3   
6           (W) 버켄스탁 보스턴 소프트 풋베드 스웨이드 레더 핑크 클레이 - 내로우        50         3   
7                                 아디다스 아딜렛 22 그레이 파이브        35         6   
8                          (W) 헌터 플레이 쇼트 레인 부츠 징크 그레이        64         4   
9                                     조던 1 로우 OG 스타피쉬        63         4   
10                    나이키 줌X 인빈시블 런 플라이니트 3 화이트 포톤더스트        55         3   
11  우포스 x 타카히로미야시타 더 솔로이스트 더 데이 더 홀 월드 웬 어웨이 플립-플랍 블랙         9         0   

In [4]:
print(len(result_df))

13


In [5]:
print(model_list[57:])

114    온 러닝 x 포스트 아카이브 팩션 (파프) 클라우드몬스터 2 문더스트 초크
115                          (W) 호카 호파라 2 쉬프팅 샌드
116                 (W) 나이키 P-6000 코코넛 밀크 메탈릭 실버
117                    나이키 줌X 스트릭플라이 화이트 클리어 제이드
118                 아식스 x 언어펙티드 젤 카야노 14 갤럭시 화이트
Name: model, dtype: object


In [6]:
print(len(model_list[57:]))

5


In [9]:
print(model_list)

57                 나이키 P-6000 프리미엄 한글날 포톤 더스트 카키
58                 살로몬 RX 슬라이드 3.0 아몬드 밀크 알로에 워시
59                              아디다스 슈퍼스타 트리플 블랙
60                      나이키 에어포스 1 미드 '07 WB 플랙스
61                조던 1 x 트래비스 스캇 레트로 하이 OG SP 모카
                         ...                    
114    온 러닝 x 포스트 아카이브 팩션 (파프) 클라우드몬스터 2 문더스트 초크
115                          (W) 호카 호파라 2 쉬프팅 샌드
116                 (W) 나이키 P-6000 코코넛 밀크 메탈릭 실버
117                    나이키 줌X 스트릭플라이 화이트 클리어 제이드
118                 아식스 x 언어펙티드 젤 카야노 14 갤럭시 화이트
Name: model, Length: 62, dtype: object


In [12]:
i = 0
for model in model_list :
    i += 1
    if model == '푸마 스피드캣 OG 스파르코 블랙 화이트':
        print(i)
    

14
